<a href="https://colab.research.google.com/github/prasanth-ntu/gpt-related/blob/main/YT_OpenAI_Functions_%2B_Finance_checker_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# README


Source: https://docs.google.com/document/d/1euGq2hO5DN6dy_-bwQhJdY032AGlPNwR61LLjj2mSyw/edit (shared in Meetup)

# Installations

In [3]:
# !pip -q install langchain openai tiktoken yfinance
!pip show langchain

Name: langchain
Version: 0.0.216
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/prasanth.thangavel/.pyenv/versions/3.10.5/envs/fastai_related/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [4]:
import json
import os

In [5]:
with open('config.json', 'r') as f:
    config = json.load(f)
os.environ["OPENAI_API_KEY"] = config['openai_api_key']

## Understanding OpenAI Functions

### OpenAI own example

Source: https://platform.openai.com/docs/guides/gpt/function-calling

In [45]:
import openai
import json

In [46]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [47]:
get_current_weather(location="Singapore", unit="fahrenheit")

'{"location": "Singapore", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [51]:
def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response, messages

In [52]:
second_response, messages = run_conversation()

In [53]:
print (second_response)

{
  "id": "chatcmpl-7VyJXoxSlix0Z32yEHe6GOv4JWyPW",
  "object": "chat.completion",
  "created": 1687854371,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Boston is currently sunny and windy with a temperature of 72 degrees."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 67,
    "completion_tokens": 17,
    "total_tokens": 84
  }
}


In [61]:
for message in messages:
    print (message)

{'role': 'user', 'content': "What's the weather like in Boston?"}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n\"location\": \"Boston\"\n}"
  }
}
{'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "Boston", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}'}


### Simple example

In [6]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    }
]

### Setting up YFinance


In [65]:
import yfinance as yf

def get_stock_price(symbol):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return round(todays_data['Close'][0], 2)

# use the function
print(get_stock_price('AAPL'))

185.27


In [66]:
# use the function
print(get_stock_price('GOOG'))

119.09


### Setting up tools

In [9]:
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from typing import Optional, Type

from pydantic import BaseModel, Field

In [10]:
# # from langchain.tools import DuckDuckGoSearchTool
# from langchain.agents import Tool
# from langchain.tools import BaseTool

# # search = DuckDuckGoSearchTool()
# # # defining a single tool
# # tools = [
# #     Tool(
# #         name = "search",
# #         func=search.run,
# #         description="useful for when you need to answer questions about current events. You should ask targeted questions"
# #     )
# # ]

In [11]:

class StockPriceCheckInput(BaseModel):
    """Input for Stock price check."""

    stockticker: str = Field(..., description="Ticker symbol for stock or index")

In [12]:
class StockPriceTool(BaseTool):
    name = "get_stock_ticker_price"
    description = "Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API"

    def _run(self, stockticker: str):
        # print("i'm running")
        price_response = get_stock_price(stockticker)

        return price_response

    def _arun(self, stockticker: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockPriceCheckInput


In [13]:
# StockPriceTool = Tool(
#     name='Get Stock Ticker price',
#     func= get_stock_price,
#     description="Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API"
# )

## LangChain doing it Manualy

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage, FunctionMessage

In [15]:
model = ChatOpenAI(model="gpt-3.5-turbo-0613")

In [16]:
from langchain.tools import MoveFileTool, format_tool_to_openai_function

In [17]:
{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'title': 'FileMoveInput',
                'description': 'Input for MoveFileTool.',
                'type': 'object',
                'properties': {'source_path': {'title': 'Source Path',
                                               'description': 'Path of the file to move',
                                               'type': 'string'},
                               'destination_path': {'title': 'Destination Path',
                                               'description': 'New path for the moved file',
                                               'type': 'string'}},
                'required': ['source_path', 'destination_path']}}

{'name': 'move_file',
 'description': 'Move or rename a file from one location to another',
 'parameters': {'title': 'FileMoveInput',
  'description': 'Input for MoveFileTool.',
  'type': 'object',
  'properties': {'source_path': {'title': 'Source Path',
    'description': 'Path of the file to move',
    'type': 'string'},
   'destination_path': {'title': 'Destination Path',
    'description': 'New path for the moved file',
    'type': 'string'}},
  'required': ['source_path', 'destination_path']}}

In [18]:
# |MoveFileTool().args_schema.schema()

In [67]:
tools = [StockPriceTool()]
functions = [format_tool_to_openai_function(t) for t in tools]

In [68]:
functions[0]

{'name': 'get_stock_ticker_price',
 'description': 'Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API',
 'parameters': {'title': 'StockPriceCheckInput',
  'description': 'Input for Stock price check.',
  'type': 'object',
  'properties': {'stockticker': {'title': 'Stockticker',
    'description': 'Ticker symbol for stock or index',
    'type': 'string'}},
  'required': ['stockticker']}}

In [69]:
ai_message = model.predict_messages(
    messages=[HumanMessage(content='What is the price of Google stock')], 
    functions=functions
)

In [70]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_stock_ticker_price', 'arguments': '{\n  "stockticker": "GOOGL"\n}'}}, example=False)

In [71]:
ai_message.additional_kwargs['function_call']

{'name': 'get_stock_ticker_price',
 'arguments': '{\n  "stockticker": "GOOGL"\n}'}

In [72]:
import json
_args = json.loads(ai_message.additional_kwargs['function_call'].get('arguments'))
_args

{'stockticker': 'GOOGL'}

In [73]:
tool_result = tools[0](_args)
tool_result

118.34

In [74]:
FunctionMessage(name='get_stock_ticker_price',content=tool_result)

FunctionMessage(content='118.34', additional_kwargs={}, name='get_stock_ticker_price')

In [27]:
final_message = model.predict_messages(
    messages = [
        HumanMessage(content='What is the price of Google stock'),
        ai_message,
        FunctionMessage(name='get_stock_ticker_price',content=tool_result),
    ], 
    functions=functions
)

In [75]:
final_message

AIMessage(content='The current price of Google stock (GOOGL) is $118.34.', additional_kwargs={}, example=False)

## Putting it together as an Agent

In [29]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [76]:
open_ai_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True)

In [77]:
open_ai_agent.run("What is the price of Google stock?")



> Entering new  chain...

Invoking: `get_stock_ticker_price` with `{'stockticker': 'GOOGL'}`


118.34The current price of Google stock (GOOGL) is $118.34.

> Finished chain.


'The current price of Google stock (GOOGL) is $118.34.'

## OpenAI Functions Agent

### Advantages
- Better reasoning and tool selection than ReACT / Toolformer?
- Less tokens needed

### Disadvantages
- Not as easy to customize via prompt change etc if it doesn't work
- locks you code into the OpenAI way
- Still need tokens for Tool/Function descriptions

## Multiple Tools

In [32]:
from datetime import datetime, timedelta

def get_price_change_percent(symbol, days_ago):
    ticker = yf.Ticker(symbol)

    # Get today's date
    end_date = datetime.now()

    # Get the date N days ago
    start_date = end_date - timedelta(days=days_ago)

    # Convert dates to string format that yfinance can accept
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')

    # Get the historical data
    historical_data = ticker.history(start=start_date, end=end_date)

    # Get the closing price N days ago and today's closing price
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]

    # Calculate the percentage change
    percent_change = ((new_price - old_price) / old_price) * 100

    return round(percent_change, 2)

# Use the function
print(get_price_change_percent('AAPL', 30))  # for 30 days ago


4.5


In [33]:
import yfinance as yf
from datetime import datetime, timedelta

def calculate_performance(symbol, days_ago):
    ticker = yf.Ticker(symbol)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_ago)
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    historical_data = ticker.history(start=start_date, end=end_date)
    old_price = historical_data['Close'].iloc[0]
    new_price = historical_data['Close'].iloc[-1]
    percent_change = ((new_price - old_price) / old_price) * 100
    return round(percent_change, 2)

def get_best_performing(stocks, days_ago):
    best_stock = None
    best_performance = None
    for stock in stocks:
        try:
            performance = calculate_performance(stock, days_ago)
            if best_performance is None or performance > best_performance:
                best_stock = stock
                best_performance = performance
        except Exception as e:
            print(f"Could not calculate performance for {stock}: {e}")
    return best_stock, best_performance

stocks = ['AAPL', 'MSFT', 'GOOG']
days_ago = 90  # change as desired

best_stock, best_performance = get_best_performing(stocks, days_ago)
print(f"The best performing stock over the past {days_ago} days is {best_stock} with a performance of {best_performance}%")


The best performing stock over the past 90 days is MSFT with a performance of 17.4%


### Make the Tools

In [34]:
from typing import List


class StockChangePercentageCheckInput(BaseModel):
    """Input for Stock ticker check. for percentage check"""

    stockticker: str = Field(..., description="Ticker symbol for stock or index")
    days_ago: int = Field(..., description="Int number of days to look back")

class StockPercentageChangeTool(BaseTool):
    name = "get_price_change_percent"
    description = "Useful for when you need to find out the percentage change in a stock's value. You should input the stock ticker used on the yfinance API and also input the number of days to check the change over"

    def _run(self, stockticker: str, days_ago: int):
        price_change_response = get_price_change_percent(stockticker, days_ago)

        return price_change_response

    def _arun(self, stockticker: str, days_ago: int):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockChangePercentageCheckInput


# the best performing

class StockBestPerformingInput(BaseModel):
    """Input for Stock ticker check. for percentage check"""

    stocktickers: List[str] = Field(..., description="Ticker symbols for stocks or indices")
    days_ago: int = Field(..., description="Int number of days to look back")

class StockGetBestPerformingTool(BaseTool):
    name = "get_best_performing"
    description = "Useful for when you need to the performance of multiple stocks over a period. You should input a list of stock tickers used on the yfinance API and also input the number of days to check the change over"

    def _run(self, stocktickers: List[str], days_ago: int):
        price_change_response = get_best_performing(stocktickers, days_ago)

        return price_change_response

    def _arun(self, stockticker: List[str], days_ago: int):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = StockBestPerformingInput

In [35]:
tools = [StockPriceTool(),StockPercentageChangeTool(), StockGetBestPerformingTool()]

# functions = [format_tool_to_openai_function(t) for t in tools]

In [36]:
tools

[StockPriceTool(name='get_stock_ticker_price', description='Useful for when you need to find out the price of stock. You should input the stock ticker used on the yfinance API', args_schema=<class '__main__.StockPriceCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False),
 StockPercentageChangeTool(name='get_price_change_percent', description="Useful for when you need to find out the percentage change in a stock's value. You should input the stock ticker used on the yfinance API and also input the number of days to check the change over", args_schema=<class '__main__.StockChangePercentageCheckInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False),
 StockGetBestPerformingTool(name='get_best_performing', description='Useful for when you need to the performance of multiple stocks over a period. You should input a list of stock tickers used on the yfinance API and also input the

In [37]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

In [38]:
open_ai_agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.OPENAI_FUNCTIONS,
                        verbose=True)

In [39]:
open_ai_agent.run("What is the price of Google stock today?")



> Entering new  chain...

Invoking: `get_stock_ticker_price` with `{'stockticker': 'GOOGL'}`


118.34The price of Google stock today is $118.34.

> Finished chain.


'The price of Google stock today is $118.34.'

In [40]:
open_ai_agent.run("Has google's stock gone up over the past 90 days?")



> Entering new  chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'GOOGL', 'days_ago': 90}`


16.72Yes, Google's stock (GOOGL) has gone up by 16.72% over the past 90 days.

> Finished chain.


"Yes, Google's stock (GOOGL) has gone up by 16.72% over the past 90 days."

In [41]:
open_ai_agent.run("How much has google's stock gone up over the past 3 months?")



> Entering new  chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'GOOGL', 'days_ago': 90}`


16.72Google's stock has gone up by 16.72% over the past 3 months.

> Finished chain.


"Google's stock has gone up by 16.72% over the past 3 months."

In [42]:
open_ai_agent.run("Which stock out of Google, Meta and MSFT has performed best over the past 3 months?")



> Entering new  chain...

Invoking: `get_best_performing` with `{'stocktickers': ['GOOGL', 'META', 'MSFT'], 'days_ago': 90}`


('META', 35.61)The stock that has performed the best over the past 3 months out of Google, Meta, and MSFT is Meta with a return of 35.61%.

> Finished chain.


'The stock that has performed the best over the past 3 months out of Google, Meta, and MSFT is Meta with a return of 35.61%.'

In [43]:
open_ai_agent.run("How much has MSFT's stock gone up over the past 3 months?")



> Entering new  chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'MSFT', 'days_ago': 90}`


17.4Microsoft's stock (MSFT) has gone up by 17.4% over the past 3 months.

> Finished chain.


"Microsoft's stock (MSFT) has gone up by 17.4% over the past 3 months."

In [44]:
open_ai_agent.run("How much has Bitcoin gone up over the past 3 months?")



> Entering new  chain...

Invoking: `get_price_change_percent` with `{'stockticker': 'BTC-USD', 'days_ago': 90}`


7.52Bitcoin has gone up by 7.52% over the past 3 months.

> Finished chain.


'Bitcoin has gone up by 7.52% over the past 3 months.'